# Optimization of spatial infraestructure for EV charging

## II. Problem formulation
$$ min \sum_{i,j} c_{ij} \pi_{ij} $$

Subject to:

$$ \sum_{j} \pi_{ij} = q_{i} $$
$$ \sum_{i} \pi_{ij} = s_{i}$$

Typical cost function:
$$C = (c_{ij})$$
Where c is a convex function, i.e.: the distance between station location and demand location in some norm:
$$c_{ij} = c(x_{i}, y_{j}) = |x_{i} - y_{j}|$$

General declarations and functions:

In [1]:
# Install Mosek solver 
import Pkg;
Pkg.add("Mosek")
Pkg.add("MosekTools")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/Documents/transport_problems/Project.toml`
  No Changes to `~/Documents/transport_problems/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Documents/transport_problems/Project.toml`
  No Changes to `~/Documents/transport_problems/Manifest.toml`


In [7]:
using JuMP, GLPK#, Mosek

In [8]:
function get_cost_matrix(x, y, norm=2)    
    m = size(x)[1]
    n = size(y)[1]
    #Calculo los costos Cij = |xi-yj| (distancia euclidea en R^2)
    c1 = (x[:,1]*ones(1,n) - ones(m,1)*y[:,1]').^norm
    c2 = (x[:,2]*ones(1,n) - ones(m,1)*y[:,2]').^norm
    return (c1+c2).^(1/norm)
end

get_cost_matrix (generic function with 2 methods)

In [9]:
m = 100; # Demand locations count
n = 10; # Station locations count
norm = 2; # Manhattan Metric

In [16]:
x = sortslices(rand(m,2), dims = 1);
y = sortslices(rand(n,2), dims = 1);
C = get_cost_matrix(x, y, norm); #Matriz de costos
q = ones(m,1);

## III. Fixed Locations
We assume that the station locations $\{ y_j \}$ have already been selected

### A. Fixed Capacities

#### Simple discrete counterexample
$$C = \begin{bmatrix}
1 & 3\\
3 & 1
\end{bmatrix}$$

$$q = \begin{bmatrix}
1\\
2
\end{bmatrix}$$

$$s = \begin{bmatrix}
2 & 1
\end{bmatrix}$$

Then:

$$\Pi^{*} = \begin{bmatrix}
1 & 0\\
1 & 1
\end{bmatrix}$$

In [11]:
C = [1 3; 3 1]
q = [1; 2]
s = [2 1]


model = JuMP.Model(GLPK.Optimizer)

@variable(model,pi[1:2, 1:2] >=0 )
@constraint(model, sum(pi, dims=2) .== q)
@constraint(model, sum(pi, dims=1) .== s )

@objective(model,Min, sum( C.*pi ))

#print(model)
model

optimize!(model)
value.(pi)

2×2 Matrix{Float64}:
 1.0  0.0
 1.0  1.0

#### Proposition 1

Suppose:

* Más ubicaciones de demanda, que estaciones: $m \ge n$
* Demandas unitarias: $q_{i} = 1$, $i = 1, ..., m$
* Capacidad justa: $s_{j} \in Z, s_{j} > 0, \sum_{j} s_{j} = m$

Then: 

La solución óptima $\Pi^{*}$ debe ser de ceros y unos ($\pi_{ij} \in \{ 0, 1\}$)


In [17]:
#Vector de capacidades
aux = rand(n,1); #random proportion for s
s = round.(aux/sum(aux)*m); #normalization
s[argmax(s)] += (m - sum(s)); #error correction
#s

In [18]:
model = JuMP.Model(GLPK.Optimizer)

@variable(model,pi[1:m,1:n]>=0)

@constraint(model, sum(pi, dims=2) .== q)
@constraint(model, sum(pi, dims=1) .== s' )

@objective(model,Min, sum( C.*pi ))

#print(model)
model;

In [19]:
optimize!(model)
value.(pi);

In [20]:
#[TODO] Find values not in 0 or 1 to verify
find(value.(pi) != 0 || value.(pi) != 1);

LoadError: UndefVarError: find not defined

#### Case m = n

In [21]:
#Vector de capacidades
s = ones(n,1); 

In [22]:
model = JuMP.Model(GLPK.Optimizer)

@variable(model,pi[1:m,1:n]>=0)

@constraint(model, sum(pi, dims=2) .== q)
@constraint(model, sum(pi, dims=1) .== s' )

@objective(model,Min, sum( C.*pi ))

#print(model)
model;

In [23]:
optimize!(model)
value.(pi);

[TODO] ¿Verificar lo de extender una $m \times n$ a $m \times m$, ponderar la capacidad y que de como el $m = n$, para luego sumar las columnas de $\Pi^{*}$ y recuperar la original ?

### B. Free Capacities

We assume that the station locations $\{ y_j \}$ have already been selected, and their supply capacities can be arbitrarly chosen.

In [24]:
model = JuMP.Model(GLPK.Optimizer)

@variable(model,pi[1:m,1:n]>=0)

@constraint(model, sum(pi, dims=2) .== q)

@objective(model,Min, sum( C.*pi ))

#print(model)
model;

In [25]:
optimize!(model)

In [26]:
#Vector de capacidades
s = sum(value.(pi), dims=1)'

10×1 adjoint(::Matrix{Float64}) with eltype Float64:
 20.0
 17.0
 13.0
  6.0
 15.0
 19.0
  7.0
  2.0
  1.0
  0.0

### C. Constrained or penalized capacities

We assume that the station locations $\{ y_j \}$ have already been selected, and their supply capacities can be arbitrarly chosen.

[TODO] ¿Network graph?

#### Proposition 2

Suppose:

* Más ubicaciones de demanda, que estaciones: $m \ge n$
* Demandas unitarias: $q_{i} = 1$, $i = 1, ..., m$
* Capacidades topeadas (aseguran demanda): $\overline{s}_{j} \in Z, s_{j} > 0, \sum_{j} \overline{s}_{j} \ge m$

Then: 

La solución óptima $\Pi^{*}$ debe ser de ceros y unos ($\pi_{ij} \in \{ 0, 1\}$)


In [27]:
#Vector de cotas de capacidad
aux = rand(n,1); #random proportion for s
tolerance = 0.2; 
s_max = round.(aux/sum(aux)*m*(1+tolerance)) #normalization

10×1 Matrix{Float64}:
 26.0
 11.0
 28.0
 14.0
  4.0
  3.0
  7.0
  5.0
  2.0
 19.0

In [28]:
model = JuMP.Model(GLPK.Optimizer)

@variable(model,pi[1:m,1:n]>=0)

@constraint(model, sum(pi, dims=2) .== q)
@constraint(model, sum(pi, dims=1) .<= s_max' )

@objective(model,Min, sum( C.*pi ))

#print(model)
model;

In [29]:
optimize!(model)

In [30]:
value.(pi)

100×10 Matrix{Float64}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮                   
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0

#### Penalized capacities

Suppose, instead, that one is designing an installation with known locations but still has not defined the relevant capacities. Leaving them complete free may be undesirable, but one could seek to even out the allocation somewhat between locations.

With this aim, we consider a problem where a soft penalty term to the transport cost:

$$ min \sum_{ij} c_{ij} \pi_{ij} + \epsilon \sum_{j} \phi_{j}(s_{j})$$

Subject to:

$$\sum_{j} \pi_{ij} = q_{i}$$
$$\sum_{i} \pi_{ij} = s_{j}$$

In this case, we use $\phi_{j}(s) = \frac{1}{2}s^{2}$

In [ ]:
function phi(s)
    return 0.5*(s.^2)
end

In [43]:
using Ipopt
model = JuMP.Model(Ipopt.Optimizer)

@variable(model, pi[1:2,1:2]>=0)
@variable(model, s[1:2]>=0)

@constraint(model, sum(pi, dims=2) .== q)
@constraint(model, sum(pi, dims=1) .== s')

@objective(model,Min, sum( C.*pi ) + eps*sum(phi(s)))

#print(model)

LoadError: DimensionMismatch("arrays could not be broadcast to a common size; got a dimension with lengths 2 and 100")

In [ ]:
optimize!(model)
value.(pi), objective_value(model)

## IV. Location Selection
In this section we consider the situation where the location poitns are also part of the design.

### A. Free locations

[TODO] K - Means

### B. Sparse selection over a set of candidate locations

Example 1 - Verification

$$x = y = \begin{bmatrix}
1\\
2\\
5
\end{bmatrix}$$

Then:

$$C = \begin{bmatrix}
0 & 1 & 4\\
1 & 0 & 3\\
4 & 3 & 0
\end{bmatrix}$$

$$q = \begin{bmatrix}
2\\
3\\
1
\end{bmatrix}$$

$$\overline{s} = \begin{bmatrix}
s & s & s
\end{bmatrix}$$

Case 1: s = 5

In [ ]:
model = JuMP.Model(GLPK.Optimizer)

@variable(model,pi[1:2,1:2]>=0)

@constraint(model, sum(pi,dims=2).== q)
@constraint(model, sum(pi, dims=1)' .<= s_max )

@objective(model,Min, sum( c.*pi ))

print(model)

optimize!(model)
value.(pi), objective_value(model)

Case 2: s = 4

### MILP (Mixed-integer linear programming)

$$min \sum_{ij} c_{ij} \pi_{ij}$$

Subject to:

$$\sum_{j} \pi_{ij} = q_i$$
$$\sum_{i} \pi_{ij} \le b_{j}\overline{s}_{j}$$
$$\sum_{j} b_{j} \le k$$
$$b_{j} \in \{ 0, 1\}$$

Where: 

$b_{j}$ indicates whether station $y_{j}$ is active or not

$k$ is the count of active actions

In [33]:
k = 3
q = 1*ones(m,1)
s_max = 100*ones(n,1)

10×1 Matrix{Float64}:
 100.0
 100.0
 100.0
 100.0
 100.0
 100.0
 100.0
 100.0
 100.0
 100.0

In [35]:
model = JuMP.Model(GLPK.Optimizer)

@variable(model,pi[1:m,1:n]>=0)
@variable(model, b[1:n], Bin)

@constraint(model, sum(pi, dims=2).== q)
@constraint(model, sum(pi, dims=1)' .<= (b.*s_max) )
@constraint(model, sum(b) <= k)

@objective(model,Min, sum( C.*pi ))

#print(model)
model

A JuMP Model
Minimization problem with:
Variables: 1010
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 100 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 11 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 1000 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 10 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: b, pi

In [37]:
optimize!(model)
objective_value(model)

26.12402888124253

In [38]:
value.(pi)

100×10 Matrix{Float64}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮                   
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0

In [39]:
value.(b)

10-element Vector{Float64}:
 1.0
 0.0
 1.0
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0